# Global import


In [3]:
from pathlib import Path

from typing import Optional, List

from mistralai.models import OCRResponse

from mistralai import Mistral, DocumentURLChunk

from mistralai.extra import response_format_from_pydantic_model

from langchain.schema import BaseMessage, HumanMessage, SystemMessage

from app.cost.cost import CostTracker

from app.proposal_object.schemas import Devis

from app.proposal_object.path import ProposalObjectPaths

from app.proposal_object.proposal_object import ProposalObjectHandler

from config.config import env_param

from config.logger_config import logger


/Users/remillieux/Documents/Kleek/OCR/app/proposal_object/schemas.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if isinstance(row[0], str) and row[0].strip() == "Lot :":
/Users/remillieux/Documents/Kleek/OCR/app/proposal_object/schemas.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  isinstance(row[2], str)
/Users/remillieux/Documents/Kleek/OCR/app/proposal_object/schemas.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

# Proposals path


In [4]:
### Hard ###
# FILE_NAME = "Devis DE2502-580 - LABOURDETTE - "


### Medium ###
# FILE_NAME = "Lot Electricité - JUILLARD - Devis_n°OF9206"

FILE_NAME = "Devis - CHAINE CHAINE DES ARTISANS COTE SUD - DEVIS LARTIZIEN   1"

# FILE_NAME = "Devis_'OF9189'"

# FILE_NAME = "Devis DE2412-461 - GOURDON - "


### Easy ###
# FILE_NAME = "Devis - CHAINE CHAINE DES ARTISANS COTE SUD - Vouland   Seignosse"

proposal_path = ProposalObjectPaths(file_name=FILE_NAME)


# Proposal Object


# Mistral Client


In [5]:
mistral_api_key = "hSRyFLyIbO0kzD7PR948CtyhLpjUQWBY"

model = "mistral-large-2411"

client = Mistral(api_key=mistral_api_key)


# Upload doc


In [6]:
uploaded_pdf = client.files.upload(
    file={
        "file_name": FILE_NAME,
        "content": open(proposal_path.proposal_path, "rb"),
    },
    purpose="ocr",
)

signed_url = client.files.get_signed_url(file_id=uploaded_pdf.id)

print(f"Signed URL: {signed_url.url}")


Signed URL: https://mistralaifilesapiprodswe.blob.core.windows.net/fine-tune/5fda61c1-c371-40c7-906b-72b14c1b9202/b2f4f412-dd41-4f47-a726-1f050aed58d8/a4af71ca6444413f82b04504bad5fcf4.pdf?se=2025-06-13T11%3A57%3A13Z&sp=r&sv=2025-05-05&sr=b&sig=e26Fd5/FlyMLcrBdJJUqjKObYwOQ3bZJ/oxHTNF/ZrE%3D


# OCR + LLM


# OCR


In [7]:
response: OCRResponse = client.ocr.process(
    model="mistral-ocr-latest",
    document=DocumentURLChunk(document_url=signed_url.url, document_name=FILE_NAME),
    # document_annotation_format=response_format_from_pydantic_model(Devis),
    include_image_base64=False,
)

all_text = ""

for page in response.pages:
    print(page.model_dump_json(indent=2))
    all_text += page.markdown

print(f"All text extracted from the document:\n\n{all_text}")

a_out = Path(proposal_path.proposal_markdown_path)

a_out.parent.mkdir(parents=True, exist_ok=True)

a_out.write_text(all_text)

print(f"✅ PROPOSAL MARKDOWN write in {a_out.resolve()}")


{
  "index": 0,
  "markdown": "![img-0.jpeg](img-0.jpeg)\n\n**SARL PEIXOTO HENRI ET FILS**\n\nMACONNERIE CARRELAGE\n\n300 AV DE LA MAIRIE 40150 ANGRESSE\n\nTél : 05.58.43.50.76\n\nFax : 05.58.43.90.61\n\nR.C.S. : DAX448519892\n\nN.I.I. : FR448519892\n\nDate de validité : 16/01/2025\n\n|  DEVIS |  |  |   |\n| --- | --- | --- | --- |\n|  N° devis | Date | Code client |   |\n|  DC1361 | 16/12/2024 | CHAINE |   |\n\nCHAINE DES ARTISANS COTE SUD\n\n650 ROUTE DE BAYONNE\n\n40230 BENNESSE MAREMNE\n\n|  Quantité | Description | Prix unitaire HT | Total HT  |\n| --- | --- | --- | --- |\n|   | chantier MME LARTIZIEN A SEIGNOSSE |  |   |\n|  1,00 | INSTALLATION CHANTIER FF | 620,00 | 620,00  |\n|  1,00 | DEPOSE DU PLANCHER BOIS EN TERRASSE FF | 880,00 | 880,00  |\n|  3,00 | DECOUPAGE DU BARDAGE BOIS FACADE NORD FF | 320,00 | 960,00  |\n|  8,00 | DEPOSE DE L ENSEMBLE DES FENETRES ET PORTES EXTERIEURES U | 80,00 | 640,00  |\n|  12,00 | DEPOSE DES VOLETS EXTERIEURS U | 60,00 | 720,00  |\n|  30,00 | 

# LLM


In [8]:
from app.agent.agent import Agent

PROPOSAL_ANALYZER_SYSTEM_PROMPT = """Vous êtes le meilleur analyseur de devis du monde. Vous devez reconstruire le devis à partir de l'OCR. Utilisez les polygones pour trouver la place des éléments. Pour chaque produit du devis, vous devez très précisément noter son label, sa description, le lot auquel il appartient son prix HT la quantité et la TVA. Le produits peuvent être grouper par catégorie et ou contenir des sous catégories de produits."""

model = "gpt-4.1"  # "gpt-4o-mini" or "o4-mini" or "gpt-4o" or "gpt-4o-2024-08-06"

cost_tracker = CostTracker()


class ProposalAnalyzer:
    """Proposal analyzer"""

    def __init__(self, cost_tracker: CostTracker) -> None:
        self.agent: Agent = Agent(
            api_key=env_param.OPENAI_API_KEY,
            model_name=model,
            system_promt=PROPOSAL_ANALYZER_SYSTEM_PROMPT,
            nb_retry=2,
            temprature=1 if model in ["o4-mini", "o3"] else 0,
            timeout=200,
            name="proposal_analyzer",
            cost_tracker=cost_tracker,
        )

    async def analyze(self, chat_historic: List[BaseMessage]) -> Optional[Devis]:
        """Strucutre the proposal"""

        proposal_structured: Optional[Devis] = await self.agent.run(
            response_format=Devis, chat_historic=chat_historic
        )

        return proposal_structured

    async def structure_proposal(self, raw_proposal: str) -> Optional[Devis]:
        """Check if the client is spam"""

        chat_historic = [
            SystemMessage(content=PROPOSAL_ANALYZER_SYSTEM_PROMPT),
            HumanMessage(content=raw_proposal),
        ]

        proposal_structured: Optional[Devis] = await self.analyze(chat_historic)

        if proposal_structured is None:
            logger.info("PROPOSAL => No structured proposal found.")

            return None

        return proposal_structured


In [ ]:
with open(
    proposal_path.proposal_markdown_path,
    "r",
    encoding="utf-8",
) as f:
    raw_proposal = f.read()

quote_analyzer = ProposalAnalyzer(cost_tracker=cost_tracker)

proposal: Optional[Devis] = await quote_analyzer.structure_proposal(
    raw_proposal=raw_proposal
)

if proposal is None:
    logger.error("PROPOSAL => No structured proposal found.")
    exit(1)

proposal_json = proposal.model_dump_json(indent=2)

print(f"Structured proposal:\n{proposal_json}")

a_out = Path(proposal_path.proposal_object_predicted_path)

a_out.parent.mkdir(parents=True, exist_ok=True)

a_out.write_text(proposal_json)

print(f"✅ OBJECT PROPOSAL written in {a_out.resolve()}")

print(f"Cost : {cost_tracker.cost.model_dump_json(indent=2)}")


# Proposal object


In [ ]:
ProposalObjectHandler(proposal_path=proposal_path).analyze_proposal_object()


2025-06-12 13:28:23 INFO -> [app/proposal_object/proposal_object.py.proposal_object.convert_to_text.49]:                             ✅ PROPOSAL TEXT write in /Users/remillieux/Documents/Kleek/OCR/data/proposal_text/Devis - CHAINE CHAINE DES ARTISANS COTE SUD - DEVIS LARTIZIEN   1.txt
2025-06-12 13:28:23 INFO -> [app/proposal_object/proposal_object.py.proposal_object.convert_to_pdf.65]:                              ✅ PROPOSAL PDF write in /Users/remillieux/Documents/Kleek/OCR/data/proposal_pdf_object/Devis - CHAINE CHAINE DES ARTISANS COTE SUD - DEVIS LARTIZIEN   1.pdf
2025-06-12 13:28:23 INFO -> [app/proposal_object/proposal_object.py.proposal_object.convert_to_proposal_lines.94]:                   ✅ PROPOSAL LINES write in /Users/remillieux/Documents/Kleek/OCR/data/proposal_object_line/Devis - CHAINE CHAINE DES ARTISANS COTE SUD - DEVIS LARTIZIEN   1.txt
2025-06-12 13:28:23 INFO -> [app/proposal_object/proposal_object.py.proposal_object.validate_proposal_object.116]:                  

arraybcimunnm = [
    "|        PACK       |   TVA   |                                                 LABEL                                                 |                                                                                  DESCRIPTION                                                                                  | QTY | UNIT | UNIT_PRICE_HT | TOTAL_HT |",
    "|-------------------|---------|-------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----|------|---------------|----------|",
    "|     MACONNERIE    | TVA 20% |                                   CHANTIER MME LARTIZIEN A SEIGNOSSE                                  |              Travaux de démolition, dépose, terrassement, réseaux, piscine, terrasse, rehaussement dalle existante pour le 